# ALP Interface

We will automate the ALP through its COM Interface 

In [1]:
from src import alp
import os
from pathlib import Path
from tqdm import tqdm
import os
import pandas as pd
from src.pile import PipePile
import numpy as np

DEBUG:matplotlib:matplotlib data path: c:\Users\dazhong.li\AppData\Local\miniconda3\envs\UnitCPT\lib\site-packages\matplotlib\mpl-data
DEBUG:matplotlib:CONFIGDIR=C:\Users\dazhong.li\.matplotlib
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is win32
DEBUG:matplotlib:CACHEDIR=C:\Users\dazhong.li\.matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from C:\Users\dazhong.li\.matplotlib\fontlist-v330.json


We will use SetNodePY function

In [2]:
model = alp.ALP()
model.new_file('test.alw','','UnitCPT','Push-Over Analysis','CPT p-y analysis')
# model.open(r'C:\Users\dazhong.li\OneDrive - Arup\01 - Project\UnitCPT\UnitCPT\alpmodel.alw')
# model.save()

proj = model.proj
pile_dia =2.0 # 1.0m
pile_thickness = 0.05 # 50mm
pile_length = 80 # 
pile_penetration = 80# 
corrosion = 0

test_pile = PipePile(pile_dia,pile_thickness,pile_length,pile_penetration,corrosion=0)
model.set_section(1,f'Tubular Pile {pile_dia}m, thickness={pile_thickness:.3f}',1,1,test_pile.EI)

# Need to set to Specified_PY
proj.SetSoilModel(1)

DEBUG:root:Terminated - alp.exe Pid-36348
DEBUG:root:Started New file test.alw


In [3]:
df = pd.read_excel('cpt_data_resampled.xlsx')
column_str =['p0','y0','p1','y1','p2','y2','p3','y3','p4','y4','p5','y5']
df_alp = df[['SCPT_DPTH','p0','y0','p1','y1','p2','y2','p3','y3','p4','y4','p5','y5']].copy()
df_alp['Level'] = df_alp.SCPT_DPTH*-1
df_alp= df_alp.dropna()
# proj.ClearNodes()
for i, row in df_alp.iterrows():
    proj.InsertNode(row['Level'],1)
    proj.AddNodePY(*(row[column_str].values.tolist()))
df_load_disp_curve = pd.DataFrame()
force = []
disp = []
rotation = []
for f in tqdm(np.linspace(0,5000,20)):
    model.set_node_load_displacement(1,f*-1,0,0)
    model.analyse()
    force.append(f)
    disp.append(model.proj.GetNodeDisp(1))
    rotation.append(model.proj.GetNodeRotation(1))

100%|██████████| 20/20 [00:10<00:00,  1.98it/s]


In [9]:
from src import geoplot
import  plotly.graph_objects as go
fig = geoplot.GEOPlot.get_figure(rows=1,cols=2)
fig.add_trace(go.Scatter(y=force,x=disp,mode='markers+lines',line=dict(color='black'),showlegend=False),row=1,col=1)
fig.add_trace(go.Scatter(y=force,x=rotation,mode='markers+lines',line=dict(color='black'),showlegend=False),row=1,col=2)
fig.update_layout(width=799,height=400,title='M=0')
fig.update_xaxes(title='Pile Head Displacement (mm)',row=1,col=1)
fig.update_xaxes(title='Pile Head Rotation (rad)',row=1,col=2)
fig.update_yaxes(title='Applied Lateral Force at Pile Head(kN)',row=1,col=1)

In [78]:
moment = []
rotation = []
disp = []
for m in tqdm(np.linspace(0,5000,20)):
    model.set_node_load_displacement(1,0,m,0)
    model.analyse()
    moment.append(m)
    rotation.append(model.proj.GetNodeRotation(1))
    disp.append(model.proj.GetNodeDisp(1))

100%|██████████| 20/20 [00:00<00:00, 26.32it/s]


In [74]:
from src import geoplot
import  plotly.graph_objects as go

fig = geoplot.GEOPlot.get_figure()
fig.add_trace(go.Scatter(y=force,x=disp,mode='markers+lines',line=dict(color='black')))
fig.update_layout(width=799,height=400)
fig.update_xaxes(title='Pile Head Displacement (mm)')
fig.update_yaxes(title='Applied Lateral Force at Pile Head(kN)')

In [75]:
fig = geoplot.GEOPlot.get_figure()
fig.add_trace(go.Scatter(y=moment,x=rotation,mode='markers+lines',line=dict(color='black')))
fig.update_layout(width=800,height=400)
fig.update_xaxes(title='Rotation (radians)')
fig.update_yaxes(title='Bending Moment (kNm)')

# Estimate Stiffness matrices using ALP

$$
\begin{bmatrix} H\\ M\\ \end{bmatrix} = \overbrace{\begin{bmatrix} K_L & K_{LR} \\ k_{LR} & K_R \end{bmatrix}}^{\text{Stiffness Matrix}} \begin{bmatrix} \rho\\ \theta \end{bmatrix}
\Rightarrow
\begin{bmatrix} \rho\\ \theta\\ \end{bmatrix} = \begin{bmatrix} K_L & K_{LR} \\ k_{LR} & K_R \end{bmatrix}^{-1} \begin{bmatrix} H \\ M \end{bmatrix}
$$

Let's produce a plot of force-displacement curve

for f in np.linspace(0,10000,100):

In [29]:
model.plot_force_Displacement()